In [126]:
import requests
import re
import pandas as pd
from PIL import Image
import hashlib

In [73]:
def load_page():
    return requests.get('http://www.humansnotinvited.com/')

In [75]:
def get_content(page):
    return str(page.content)

In [241]:
def find_all_images(content):
    return re.findall('captcha/image.php\\?image_name=.*?&id=.', content)

In [84]:
def find_tag(content):
    return re.findall('value="(.*?)" name="category"', content)[0]

In [243]:
%%time
d = {}
n = 1
for i in range(n):
    content = get_content(load_page())
    new_images, new_tag = find_all_images(content), find_tag(content)
    d[new_tag] = d.get(new_tag, {})
    for img in new_images:
        load_img = Image.open(requests.get('http://www.humansnotinvited.com/' + img, stream=True).raw)
        hsh = hashlib.md5(load_img.tobytes()).hexdigest()
        d[new_tag][hsh] = d[new_tag].get(hsh, 0) + 1
df = pd.DataFrame(d)

Wall time: 7.94 s


In [267]:
df = pd.read_csv('nothuman.csv', index_col=0)

In [289]:
content = """
<div class="captcha">
      <div class="container">
        <div class="captcha-container">

          <div class="header">
            <p>Select all squares with <strong>kids</strong></p>
            <input type="hidden" value="kids" name="category" />
          </div>

          <div class="content">
            <div class="captcha-image" data-token="$1$W1kmA8cU$dc26k9DHnNkoWk4B2eprH." data-id="1"><img src="captcha/image.php?image_name=$1$W1kmA8cU$dc26k9DHnNkoWk4B2eprH.&id=1" alt="" /></div><div class="captcha-image" data-token="$1$5sPzqDRW$kFICV2vmmR56sephbS4e4/" data-id="5"><img src="captcha/image.php?image_name=$1$5sPzqDRW$kFICV2vmmR56sephbS4e4/&id=5" alt="" /></div><div class="captcha-image" data-token="$1$lrmgAK2I$/kPso6NSocp8E/TdXTFUD." data-id="6"><img src="captcha/image.php?image_name=$1$lrmgAK2I$/kPso6NSocp8E/TdXTFUD.&id=6" alt="" /></div><div class="captcha-image" data-token="$1$q//TBhnE$ahSgZE8dyc.dJAi5AKrUm/" data-id="6"><img src="captcha/image.php?image_name=$1$q//TBhnE$ahSgZE8dyc.dJAi5AKrUm/&id=6" alt="" /></div><div class="captcha-image" data-token="$1$AsFUXFU8$V/7gmrPWWzoCwUghmDT5c0" data-id="7"><img src="captcha/image.php?image_name=$1$AsFUXFU8$V/7gmrPWWzoCwUghmDT5c0&id=7" alt="" /></div><div class="captcha-image" data-token="$1$9ZCAXBWu$s/i2/04QszN5QwjJioRbf0" data-id="7"><img src="captcha/image.php?image_name=$1$9ZCAXBWu$s/i2/04QszN5QwjJioRbf0&id=7" alt="" /></div><div class="captcha-image" data-token="$1$SiUje852$x3qdTWVnkrfR9JWaiX6rE." data-id="3"><img src="captcha/image.php?image_name=$1$SiUje852$x3qdTWVnkrfR9JWaiX6rE.&id=3" alt="" /></div><div class="captcha-image" data-token="$1$9IzC6ykv$oz6yBPPfQVCJV8SYLX/.s1" data-id="8"><img src="captcha/image.php?image_name=$1$9IzC6ykv$oz6yBPPfQVCJV8SYLX/.s1&id=8" alt="" /></div><div class="captcha-image" data-token="$1$OMBTky8K$lFKpycFv5/ZtPj9zsupdh1" data-id="3"><img src="captcha/image.php?image_name=$1$OMBTky8K$lFKpycFv5/ZtPj9zsupdh1&id=3" alt="" /></div>          </div>
        </div>

"""

In [290]:
q = []
new_images, new_tag = find_all_images(content), find_tag(content)
for img in new_images:
    load_img = Image.open(requests.get('http://www.humansnotinvited.com/' + img, stream=True).raw)
    hsh = hashlib.md5(load_img.tobytes()).hexdigest()
    q.append(hsh)

In [291]:
answ = df.loc[q, [new_tag]] > 1
for i in range(len(answ)):
    if answ.iloc[i][new_tag]:
        print(i + 1)

3
5
8
